In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            #1st Hidden Layer
            nn.Linear(5, 128), 
            #입력 텐서의 크기가 5, 출력 텐서의 크기가 128인 모듈 생성 
            #Batch Normalization between 'Layer' and 'Activation function'
            nn.BatchNorm1d(128), 
            #입력이 128 크기, 배치 정규화 수행
            nn.ReLU(), 
            #ReLU 함수를 활성화함수로 사용!
            #Drop out after 'Activation function'
            nn.Dropout(0.1), 
            #드롭 아웃 진행!
            
            #2nd Hidden Layer
            nn.Linear(128, 256), 
            #위에서 출력 텐서 크기가 128, 그래서 이번 층은 입력 텐서 크기가 128
            nn.BatchNorm1d(256), 
            #출력 텐서 크기를 256으로 지정 -> 파라미터 256으로 설정 후 배치 정규화 수행
            nn.ReLU(), 
            nn.Dropout(0.1), 
            
            #3rd Hidden Layer
            nn.Linear(256, 128), 
            nn.BatchNorm1d(128), 
            nn.ReLU(), 
            nn.Dropout(0.1), 
            
            #Output Layer
            nn.Linear(128, 1), 
            nn.Sigmoid()
        )
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [6]:
data_set = pd.concat((train.drop(['Survived'], axis = 1), test), axis = 0)

data_set = data_set.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis = 1)
data_set = data_set.fillna(data_set.mean())

n_train = train.shape[0]
train_x, test_x = data_set[:n_train], data_set[n_train:]
train_y = train['Survived']

train_x = train_x[train_x.keys()].values
test_x = test_x[test_x.keys()].values
train_y = train_y.values

import torch.optim as optim
from torch.autograd import Variable

simple_nn = SimpleNN()
optimizer = optim.Adam(simple_nn.parameters(), lr = 0.01)
#Adam 옵티마이저 사용, 학습률 0.01 
error = nn.BCELoss()
#error는 Binary Cross Entropy!

batch_size = 99
batch_count = int(len(train_x)/batch_size)  
#미니 배치의 수를 batch_count에 저장

for epoch in range(300):
    train_loss = 0
    num_right = 0
    for i in range(batch_count):
        start = i*batch_size
        #미니 배치가 시작되는 인덱스
        end = start + batch_size
        #미니 배치의 마자막 데이터 인덱스
        tensor_x = torch.FloatTensor(train_x[start:end])
        tensor_y = torch.FloatTensor(train_y[start:end]).reshape(-1, 1)
        #i 번째 미니 배치에 대한 x, y데이터 생성, y의 경우 reshape로 확실하게 크기 지정 
        
        optimizer.zero_grad()
        #그래디언트 초기화
        output = simple_nn(tensor_x)
        #층의 출력값
        loss = error(output, tensor_y)
        loss.backward()
        #역전파 
        optimizer.step()
        #가중치 업데이트 
        
        train_loss += loss.item() * batch_size
        #학습 오차를 미니배치에 따라 더해감 
        result = [1 if out >= 0.5 else 0 for out in output]
        #output 값의 임곗값을 0.5로 두고 그 이상이면 1, 미만이면 0을 반환
        num_right += np.sum(np.array(result) == train_y[start:end])
    train_loss = train_loss / len(train_x)
    accuracy = num_right / len(train_x)
    
    if epoch % 25 == 0:
        print('Loss: {} Accuracy: {}% Epoch:{}'.format(train_loss, accuracy, epoch))
        
print('Training Ended')

Loss: 0.6525094244215224 Accuracy: 0.6386083052749719% Epoch:0
Loss: 0.5250979165236155 Accuracy: 0.7575757575757576% Epoch:25
Loss: 0.47150171796480816 Accuracy: 0.7856341189674523% Epoch:50
Loss: 0.3945884903271993 Accuracy: 0.8170594837261503% Epoch:75
Loss: 0.355927179257075 Accuracy: 0.8361391694725028% Epoch:100
Loss: 0.3714137176672618 Accuracy: 0.8294051627384961% Epoch:125
Loss: 0.3206174754434162 Accuracy: 0.8507295173961841% Epoch:150
Loss: 0.2851562119192547 Accuracy: 0.8810325476992144% Epoch:175
Loss: 0.27909259498119354 Accuracy: 0.8799102132435466% Epoch:200
Loss: 0.24603596164120567 Accuracy: 0.8945005611672279% Epoch:225
Loss: 0.22723451422320473 Accuracy: 0.9057239057239057% Epoch:250
Loss: 0.2204576548602846 Accuracy: 0.9023569023569024% Epoch:275
Training Ended


In [7]:
#test 데이터에 대한 예측 수행, 결과 파일 생성 
tensor_test_x = torch.FloatTensor(test_x)
with torch.no_grad():
    test_output = simple_nn(tensor_test_x)
    result = np.array([1 if out >= 0.5 else 0 for out in test_output])
    submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived' : result})
    submission.to_csv('submission.csv', index = False)